# Exploratory Data Analysis - UIUC Airfoil Database

This notebook tries to analysize and explore all the airfoils from the UIUC database:
https://m-selig.ae.illinois.edu/ads/coord_database.html

In [1]:
import os
import subprocess
import time
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from bs4 import BeautifulSoup
import re
try:
    import urllib.request as urllib2
except:
    import urllib2

### Download all UIUC airfoils

We do web scrapping and extract all the airfoils from the database using `BeautifulSoup` and save them into `coordinate_data` folder.

In [3]:
# Set the `download = True` to download
download = False

if download:
    # Base filepath
    base_filept = 'https://m-selig.ae.illinois.edu/ads/'

    # Open the webpage
    html_page = urllib2.urlopen('https://m-selig.ae.illinois.edu/ads/coord_database.html')
    soup = BeautifulSoup(html_page, 'lxml')

    # Loop over airfoil .dat files and save each
    ind = 1
    airfoil_names = []
    for link in soup.find_all('a', attrs={'href': re.compile('\.dat', re.IGNORECASE)}):
        airfoil_names.append(link.get('href').rsplit('/',1)[-1])
        urllib2.urlretrieve(base_filept+link.get('href'), 'coordinate_data/'+link.get('href').rsplit('/',1)[-1])
        print(f'Saving airfoil {ind} ...')
        ind = ind + 1

    with open('names_list/airfoil_names.txt', 'w') as f:
        for ind in range(len(airfoil_names)):
            f.write(f'{airfoil_names[ind]}\n')

### Preprocess the airfoils

This step is necessary so that the coordinates can be imported into XFOIl. The convention is to sort the points via **TE-upper-LE-lower-TE**

In [4]:
# Set `preprocess = True` to conduct the procedure
preprocess = False

if preprocess:
    # Read airfoil names
    with open('names_list/airfoil_names.txt', 'r') as f:
        names_list = f.readlines()

    for i in range(len(names_list)):
        names_list[i] = names_list[i][:-1]

    for ind in range(len(names_list)):
        print(f'{ind+1}. Preprocessing {names_list[ind]} ...')

    # Loading raw coordinates with various formats
    try:
        coord = np.genfromtxt(f'coordinate_data/{names_list[ind]}')
    except ValueError:
        try:
            coord = np.genfromtxt(f'coordinate_data/{names_list[ind]}', skip_header=1)
        except ValueError:
            try:
                coord = np.genfromtxt(f'coordinate_data/{names_list[ind]}', skip_header=2)
            except ValueError:
                try:
                    coord = np.genfromtxt(f'coordinate_data/{names_list[ind]}', skip_header=3)
                except ValueError:
                    coord = np.genfromtxt(f'coordinate_data/{names_list[ind]}', skip_header=3, skip_footer=1)

    # Remove nan
    if np.isnan(coord[0, 0]):
        coord = coord[1:]

    # Remove number of points
    if coord[0, 0] > 1.0:
        coord = coord[1:]

    # Make the XFOIL format
    if coord[0, 0] < 0.1: # Is the point LE?
        for i in range(len(coord)): # Sweeping to find break point
            if i != len(coord)-1:
                if coord[i, 0] > coord[i+1, 0]:
                    coord_n = np.concatenate((coord[:i+1][::-1], coord[i+1:]), axis=0)
    else:
        coord_n = coord.copy()

    # Saving processed coordinates
    np.savetxt(f'processed_coordinates/{names_list[ind]}', coord_n)

### Analyze the airfoils (create polar data)

Analyze all the airfoils automatically using XFOIL with the following conditions (and leave the rest default): <br>

1. `ASeq = [-2, 10, 0.5]` <br>
The AoA sweep from -2 to 10 with 0.5 increment.
2. `Re = 3.5e6` <br>
The Reynold number is 3.5 million.
3. `Mach = 0.117` <br>
The Mach number is 0.117 or around 40 m/s.
4. `n_iter = 100` <br>
The number of iteration in XFOIL setting equals to 100.

In [6]:
# Set `analze == True` to perform the process
analyze = False

if analyze:
    # Read inputs
    with open('names_list/airfoil_names.txt', 'r') as f:
        names_list = f.readlines()

    for i in range(len(names_list)):
        names_list[i] = names_list[i][:-1]

    t_init = time.time()

    for ind in range(len(names_list)):
        print(f'Analyzing {names_list[ind]} ...')

        airfoil_name = names_list[ind]
        alpha_i = -2
        alpha_f = 10
        alpha_step = 0.5
        Re = 3.5e6
        Mach = 0.117
        n_iter = 100

        # XFOIL Input
        if os.path.exists(f'polar_data/{names_list[ind]}'):
            os.remove(f'polar_data/{names_list[ind]}')

        with open('input_file.in', "w") as f:
            f.write(f'LOAD processed_coordinates/{airfoil_name}\n')
            f.write(f'{airfoil_name}\n')
            f.write('PANE\n')
            f.write('OPER\n')
            f.write(f'Visc {Re}\n')
            f.write(f'Mach {Mach}\n')
            f.write('PACC\n')
            f.write(f'polar_data/{names_list[ind]}\n\n')
            f.write(f'Iter {n_iter}\n')
            f.write(f'ASeq {alpha_i} {alpha_f} {alpha_step}\n')
            f.write('\n')
            f.write('quit\n')

        subprocess.call('xfoil < input_file.in', shell=True)

    t_final = time.time()
    np.savetxt('analysis_time.dat', [t_final-t_init])

### Importing polar data

`airfoil_names.txt` contains the names of all the airfoils from the database.<br>
`problematic_airfoil.txt` contains the names of all the airfoils that have problems with XFOIL.<br>
`not_converged.txt` contains the names of all the airfoils that XFOIL does not converge on.<br>

In [7]:
# Read
with open('names_list/airfoil_names.txt', 'r') as f:
    names_list = f.readlines()
with open('names_list/problematic_airfoil.txt', 'r') as f:
    problematic_list = f.readlines()
with open('names_list/not_converged.txt', 'r') as f:
    empty_list = f.readlines()

for i in range(len(names_list)):
    names_list[i] = names_list[i][:-1]

for i in range(len(problematic_list)):
    problematic_list[i] = problematic_list[i][:-1]
    
for i in range(len(empty_list)):
    empty_list[i] = empty_list[i][:-1]

# List of airfoils with successful analysis
done_list = list(set(names_list) - set(problematic_list) - set(empty_list))

# Importing polar data and append it into a list
polar_data = []
for ind in range(len(done_list)):
    polar_data.append(np.genfromtxt(f'polar_data/{done_list[ind]}', skip_header=12))

print(f'Converged airfoils: {len(done_list)}')
print(f'Not-converged airfoils: {len(empty_list)}')
print(f'Problematic airfoils: {len(problematic_list)}')
print(f'Total airfoils: {len(done_list) + len(empty_list) + len(problematic_list)}')

Converged airfoils: 1360
Not-converged airfoils: 198
Problematic airfoils: 34
Total airfoils: 1592


### Create a summary for `AoA = [-2, 10, 0.5]` in csv files and a single csv file

We create separate summaries for each AoA.

In [23]:
AoA_list = np.arange(-2.0, 10.5, 0.5)

for AoA in AoA_list:
    summary_np = np.zeros((len(done_list), 7))

    for ind in range(len(done_list)):
        try: # Find data at AoA == AoA_list
            idx = np.where(polar_data[ind][:,0]==AoA)[0][0]
            summary_np[ind] = polar_data[ind][idx]
        except IndexError: # airfoils that don't have the data at AoA == AoA_list
            summary_np[ind] = [AoA, np.nan, np.nan, np.nan, np.nan, np.nan, np.nan]

    summary_df = pd.DataFrame(data=summary_np,
                                columns=['alpha', 'CL', 'CD', 'CDp', 'CM', 'Top_Xtr', 'Bot_Xtr'])

    summary_df['name'] = done_list
    name_col = summary_df.pop('name')
    summary_df.insert(0, 'name', name_col)

    summary_df.to_csv(f'summary/summary_AoA_{AoA}.csv', index=False)

Now we create a summary in a single csv file named `summary/summary_all.csv`

In [33]:
# Importing csv files into DataFrames
AoA_list = np.arange(-2.0, 10.5, 0.5)

df_list = []

for AoA in AoA_list:
    df_list.append(pd.read_csv(f'summary/summary_AoA_{AoA}.csv'))

In [100]:
# Instantiate a python dictionary and lists
dict_all = {}
name_list = []
indicator_list = []
alpha_list = []
value_list = []

# Loop over for name
for ind in range(len(df_list[0])):
    name_list.append(df_list[0].name[ind])
    name_list.append(df_list[0].name[ind])
    name_list.append(df_list[0].name[ind])
    name_list.append(df_list[0].name[ind])
    name_list.append(df_list[0].name[ind])
    name_list.append(df_list[0].name[ind])
    name_list.append(df_list[0].name[ind])
name_list = name_list * len(AoA_list)

# Loop over for indicator
for ind in range(len(df_list[0])):
    indicator_list.append(df_list[0].columns[2])
    indicator_list.append(df_list[0].columns[3])
    indicator_list.append(df_list[0].columns[4])
    indicator_list.append(df_list[0].columns[5])
    indicator_list.append(df_list[0].columns[6])
    indicator_list.append(df_list[0].columns[7])
    indicator_list.append('L_by_D')
indicator_list = indicator_list * len(AoA_list)

# Loop over for alpha
for AoA in AoA_list:
    for ind in range(7 * len(df_list[0])):
        alpha_list.append(AoA)

# Loop over for value
for _, df in enumerate(df_list):
    for ind in range(len(df_list[0])):
        for indicator in indicator_list[:7]:
            if indicator == 'L_by_D':
                value = df.iloc[ind]['CL']/df.iloc[ind]['CD']
                value_list.append(value)
            else:
                value = df.iloc[ind][indicator]
                value_list.append(value)

In [101]:
len(name_list), len(alpha_list), len(indicator_list), len(value_list)

(238000, 238000, 238000, 238000)

In [104]:
# Create DataFrame and save to a csv file
dict_all = {}
dict_all['Airfoil Name'] = name_list
dict_all['Indicator Name'] = indicator_list
dict_all['Alpha'] = alpha_list
dict_all['Value'] = value_list

df_all = pd.DataFrame(dict_all)
df_all.to_csv('summary/summary_all.csv', index=False)

### Create dash plotly figure for exploration

We explore the data at `AoA = [0.0, 1.0, 2.0, 3.0]` for successfully converged airfoils.

In [14]:
# AoA = 0.0
summary = pd.read_csv('summary/summary_AoA_0.0.csv')
summary.dropna(inplace=True)
print(f'Total processed airfoils: {len(summary)}')

Total processed airfoils: 1278


In [ ]:
# AoA = 1.0

In [ ]:
# AoA = 2.0

In [ ]:
# AoA = 3.0

In [ ]:
fig = go.Figure()

fig.add_trace(
    go.Scatter(mode='markers',
               x=-data['CL'][data['is_pareto']=='yes'],
               y=data['CD'][data['is_pareto']=='yes'],
               name='Non-dominated design',
               marker=dict(
                    color='red',
                    size=10,
                    symbol='circle'
                ),
               hoverinfo='none'
        )
    )